In [2]:
import pandas as pd
import numpy as np

In [3]:
# Chargement des DataSets
fps_df = pd.read_csv('../../../Data/Regression/fps in video games/fps-in-video-games_clean.csv')
heart_df = pd.read_csv('../../../Data/Classification/Indicators of Heart Disease/heart_2022_no_nans_clean.csv')
retail_df = pd.read_csv('../../../Data/Clustering/retail Data/retail_data_clean.csv')

/var/folders/qc/m20c7_k95j52q7xcg6lwyd7w0000gn/T/ipykernel_15480/3500090452.py:2: DtypeWarning: Columns (25,30) have mixed types. Specify dtype option on import or set low_memory=False.
  fps_df = pd.read_csv('../../../Data/Regression/fps in video games/fps-in-video-games_clean.csv')


In [4]:
def feature_accuracy(df, pollution_levels, target_column=None, noise_factor=1):
    df_dict = {}

    for pollution_percentage in pollution_levels:
        df_polluted = df.copy()

        # Pollution des données catégoriques, en excluant la colonne cible
        categorical_columns = [col for col in df_polluted.select_dtypes(include=['object', 'category']).columns
                               if col != target_column]
        for col in categorical_columns:
            # Sélectionner un sous-ensemble de lignes à polluer
            mask = np.random.rand(len(df_polluted)) < pollution_percentage
            unique_values = df_polluted[col].unique()

            # Pour chaque ligne sélectionnée, changer la valeur par une autre valeur aléatoire différente
            df_polluted.loc[mask, col] = df_polluted.loc[mask, col].apply(
                lambda x: np.random.choice(unique_values[unique_values != x])
            )

        # Pollution des données numériques, en excluant la colonne cible
        numeric_columns = [col for col in df_polluted.select_dtypes(include=[np.number]).columns
                           if col != target_column]
        for col in numeric_columns:
            # Sélectionner un sous-ensemble de lignes à polluer
            mask = np.random.rand(len(df_polluted)) < pollution_percentage

            # Vérifier si la colonne ne contient que des valeurs entières (même si elles sont de type float)
            is_integer_column = df_polluted[col].dropna().apply(lambda x: x == int(x)).all()

            if is_integer_column:
                # Ajouter du bruit gaussien aux valeurs sélectionnées
                noise = np.random.normal(0, df_polluted[col].std() * noise_factor, size=len(df_polluted[mask]))
                df_polluted.loc[mask, col] += noise

                # Arrondir à l'entier le plus proche
                df_polluted[col] = df_polluted[col].round(0)  # Arrondir à 0 décimales (donc à l'entier)
            else:
                # Récupérer le nombre de décimales de la première valeur non nulle pour garder la précision
                precision = df_polluted[col].dropna().apply(
                    lambda x: len(str(x).split('.')[1]) if '.' in str(x) else 0
                ).max()

                # Ajouter du bruit gaussien aux valeurs sélectionnées
                noise = np.random.normal(0, df_polluted[col].std() * noise_factor, size=len(df_polluted[mask]))
                df_polluted.loc[mask, col] += noise

                # Arrondir à la précision originale
                df_polluted[col] = df_polluted[col].round(precision)

        # Ajouter le DataFrame pollué au dictionnaire avec le pourcentage de pollution comme clé
        df_dict[f"{int(pollution_percentage * 100)}%"] = df_polluted

    return df_dict

In [5]:
# Liste des pourcentages de valeurs manquantes à introduire
pollution_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

# Génération des DataFrames avec valeurs manquantes
fps_df_with_feature_accuracy = feature_accuracy(fps_df, pollution_levels,'FPS')
heart_df_with_feature_accuracy = feature_accuracy(heart_df, pollution_levels,'HadHeartAttack')
retail_df_with_feature_accuracy = feature_accuracy(retail_df, pollution_levels)

/var/folders/qc/m20c7_k95j52q7xcg6lwyd7w0000gn/T/ipykernel_15480/1015871928.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 0.01467087  0.36297171 -0.61518537 ...  1.47184361  0.52555169
  1.25100318]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_polluted.loc[mask, col] += noise
/var/folders/qc/m20c7_k95j52q7xcg6lwyd7w0000gn/T/ipykernel_15480/1015871928.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-0.79372004 -0.07114148 -0.17164539 ...  1.03763643  0.82458276
  1.52385086]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_polluted.loc[mask, col] += noise
/var/folders/qc/m20c7_k95j52q7xcg6lwyd7w0000gn/T/ipykernel_15480/1015871928.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise a

In [6]:
# Enregistrement des DataFrames générés
for key, value in fps_df_with_feature_accuracy.items():
    value.to_csv(f"../../../Data/Regression/fps in video games/Feature Accuracy/fps_{key}.csv", index=False)

for key, value in heart_df_with_feature_accuracy.items():
    value.to_csv(f"../../../Data/Classification/Indicators of Heart Disease/Feature Accuracy/heart_{key}.csv", index=False)

for key, value in retail_df_with_feature_accuracy.items():
    value.to_csv(f"../../../Data/Clustering/retail Data/Feature Accuracy/retail_{key}.csv", index=False)